In [1]:
import meshio
import numpy as np
import time

from LECMesh import LECMesh

In [10]:
infile = "australia/data/TAS.vtk"
outfile = "outputs/costed_tas.vtk"
mesh = meshio.read(infile)

# Setup the output data structure:
mesh.point_data['cost'] = np.zeros_like(mesh.point_data['Z'])

In [3]:
lm = LECMesh(mesh, max_fuel = 2500)

In [4]:
points_above_sealevel = np.nonzero(mesh.point_data['Z'] >= 0)[0]
print("Total starting points available: ", points_above_sealevel.shape[0], flush=True)

Total starting points available:  1093


In [5]:
def process_points_with_feedback(points):
    """
    This function is overly complicated, only because it's nice to see progress
    """
    all_costs = []
    start = 0
    inc = 50
    stop = inc
    
    while start < len(points):
        start_time = time.time()
        
        # the real work is being done here
        costs = list(map(lm.get_dist_from_point, points[start:stop]))
        # you could replace this whole function with just:
        #     return list(map(lm.get_dist_from_point, points))
        # I think...
        
        print("points: {: 7d} to {: 7d} took {:.3f} s. % complete: {:.3f}".format(
                                                                                 start, 
                                                                                 stop, 
                                                                                 time.time() - start_time, 
                                                                                 (stop/(len(points)-1)) * 100), flush=True)

        # Save the data
        all_costs.extend(costs)
        
        # move to the next chunk of data
        start += inc
        stop += inc
        if stop >= len(points):
            stop = len(points)-1
            
    return all_costs

In [6]:
all_costs = process_points_with_feedback(points_above_sealevel)

points:       0 to      50 took 22.288 s. % complete: 4.579
points:      50 to     100 took 22.427 s. % complete: 9.158
points:     100 to     150 took 23.171 s. % complete: 13.736
points:     150 to     200 took 22.885 s. % complete: 18.315
points:     200 to     250 took 23.047 s. % complete: 22.894
points:     250 to     300 took 22.749 s. % complete: 27.473
points:     300 to     350 took 22.516 s. % complete: 32.051
points:     350 to     400 took 21.906 s. % complete: 36.630
points:     400 to     450 took 23.194 s. % complete: 41.209
points:     450 to     500 took 21.544 s. % complete: 45.788
points:     500 to     550 took 19.023 s. % complete: 50.366
points:     550 to     600 took 21.335 s. % complete: 54.945
points:     600 to     650 took 22.808 s. % complete: 59.524
points:     650 to     700 took 23.381 s. % complete: 64.103
points:     700 to     750 took 20.741 s. % complete: 68.681
points:     750 to     800 took 21.311 s. % complete: 73.260
points:     800 to     850

## Output

Show some of the results:

In [7]:
print(all_costs[:100])

[(22, 55418082), (43, 9648), (47, 60083785), (54, 32523020), (58, 43708628), (111, 61688795), (116, 59654844), (125, 35304426), (126, 56257214), (127, 58083437), (128, 57878021), (129, 57612143), (130, 58057943), (131, 57092540), (132, 57413375), (133, 60022476), (134, 59843374), (135, 59847742), (136, 60613939), (137, 60043852), (138, 59326592), (139, 57150938), (140, 58528761), (141, 57608035), (142, 58182366), (143, 58456637), (144, 59317098), (145, 59203800), (146, 61267146), (147, 51705534), (148, 51745217), (149, 52223910), (150, 50969998), (151, 52949049), (152, 51734716), (153, 50563743), (154, 50265524), (155, 55285205), (156, 55737761), (157, 56558502), (158, 59063904), (159, 60923238), (160, 60341105), (161, 65257824), (162, 65527009), (163, 66723367), (164, 67990463), (165, 68435922), (166, 69822023), (167, 72237048), (168, 73946974), (169, 71835785), (170, 73406668), (171, 71749703), (172, 72113603), (173, 69503225), (174, 69114057), (175, 68392920), (176, 66410744), (177,

Write the data out to disk

In [11]:
for i in all_costs:
    mesh.point_data['cost'][i[0]] = i[1]
meshio.write(outfile, mesh)